### Instance Methods

We can create other instance methods. They work the same way as `__init__`, but don't get called automatically.

In [1]:
class Circle:
    def __init__(self, radius):
        self.radius = radius
        
    def area(self):
        print('area called...')

In [2]:
c = Circle(1)

In [3]:
c.radius

1

So we have a circle with a radius of `1`, and we can also call the `area` method:

In [4]:
c.area()

area called...


The `self` paremter is automatically passed by Python, using whatever is to the **left** of the dot, so the instance `c` in this case.

Now, let's implement that `area` method:

In [5]:
import math

class Circle:
    def __init__(self, radius):
        self.radius = radius
        
    def area(self):
        return math.pi * self.radius ** 2

You'll notice how we are accessing the `radius` attribute of the instance in the `area` instance method. Let's see how it works.

In [6]:
c = Circle(1)

In [7]:
c.radius

1

In [8]:
c.area()

3.141592653589793

So we can assign state to objects simply by setting attributes on the object, and we can implement functionality by creating instance methods.

These methods are just functions that end up being bound to the instance they are called "from", so we can specify additional paramters as well if we want to.

In [9]:
class Circle:
    def __init__(self, center_x, center_y, radius):
        self.center = (center_x, center_y)
        self.radius = radius
        
    def area(self):
        return math.pi * self.radius ** 2
    
    def translate(self, x, y):
        self.center = (self.center[0] + x, self.center[1] + y)
        
    def scale(self, factor):
        self.radius = self.radius * factor

In [10]:
c = Circle(0, 0, 2)

In [11]:
c.center

(0, 0)

In [12]:
c.radius

2

In [13]:
c.translate(1, 2)

In [14]:
c.center

(1, 2)

In [15]:
c.radius

2

In [16]:
c.scale(3)

In [17]:
c.radius

6

Now we know how to create and initialize classes, as well as implement methods that can be used to calculate things, or even modify the internal state of the object.

This allows us to "package" up related functionality neatly in custom classes. (This is called *encapsulation*)

For example, we looked at an example earlier that loaded forex data from a text file: 

In [18]:
file_name = 'DEXUSEU.csv'

In [19]:
with open(file_name) as f:
    for _ in range(5):
        print(next(f))

DATE,DEXUSEU

2015-04-03,1.0990

2015-04-06,1.1008

2015-04-07,1.0850

2015-04-08,1.0818



We ended up writing some code to load the data up. We're going to do the same thing here, but we'll also want to provide the data parsed into `datetime` and `Decimal` objects, as well as provide the headers of the data.

Let's just write some simple scripts first to see how we can start tackling this problem.

First we have to load the data, so we'll use the `csv` module for that:

In [20]:
import csv

In [21]:
with open(file_name) as f:
    reader = csv.reader(f)
    
    headers = next(reader)
    
    data = list(reader)

In [22]:
print(headers)

['DATE', 'DEXUSEU']


In [23]:
print(data[:5])

[['2015-04-03', '1.0990'], ['2015-04-06', '1.1008'], ['2015-04-07', '1.0850'], ['2015-04-08', '1.0818'], ['2015-04-09', '1.0671']]


We will want to parse the dates into `date` objects, for that we'll use the `strptime` function:

In [24]:
from datetime import datetime

In [25]:
datetime.strptime('2015-04-03', '%Y-%m-%d').date()

datetime.date(2015, 4, 3)

Also, we'll want to use `Decimal` objects for the values:

In [26]:
from decimal import Decimal

In [27]:
Decimal('1.0990')

Decimal('1.0990')

Ok, now we have all the pieces we need, let's start packaging this up into a class:

In [28]:
class Forex:
    def __init__(self, file_name):
        with open(file_name) as f:
            reader = csv.reader(f)
            
            self.headers = next(reader)
            self.data = list(reader)

In [29]:
forex = Forex(file_name)

In [30]:
forex.headers

['DATE', 'DEXUSEU']

In [31]:
forex.data

[['2015-04-03', '1.0990'],
 ['2015-04-06', '1.1008'],
 ['2015-04-07', '1.0850'],
 ['2015-04-08', '1.0818'],
 ['2015-04-09', '1.0671'],
 ['2015-04-10', '1.0598'],
 ['2015-04-13', '1.0582'],
 ['2015-04-14', '1.0672'],
 ['2015-04-15', '1.0596'],
 ['2015-04-16', '1.0742'],
 ['2015-04-17', '1.0780'],
 ['2015-04-20', '1.0763'],
 ['2015-04-21', '1.0758'],
 ['2015-04-22', '1.0729'],
 ['2015-04-23', '1.0803'],
 ['2015-04-24', '1.0876'],
 ['2015-04-27', '1.0892'],
 ['2015-04-28', '1.0979'],
 ['2015-04-29', '1.1174'],
 ['2015-04-30', '1.1162'],
 ['2015-05-01', '1.1194'],
 ['2015-05-04', '1.1145'],
 ['2015-05-05', '1.1174'],
 ['2015-05-06', '1.1345'],
 ['2015-05-07', '1.1283'],
 ['2015-05-08', '1.1241'],
 ['2015-05-11', '1.1142'],
 ['2015-05-12', '1.1240'],
 ['2015-05-13', '1.1372'],
 ['2015-05-14', '1.1368'],
 ['2015-05-15', '1.1428'],
 ['2015-05-18', '1.1354'],
 ['2015-05-19', '1.1151'],
 ['2015-05-20', '1.1079'],
 ['2015-05-21', '1.1126'],
 ['2015-05-22', '1.1033'],
 ['2015-05-25', '.'],
 ['201

This is a good start, but we still want to parse the date and values into `date` and `Decimal` objects:

In [32]:
class Forex:
    def __init__(self, file_name):
        with open(file_name) as f:
            reader = csv.reader(f)
            
            self.headers = next(reader)
            self.data = [
                (datetime.strptime(date, '%Y-%m-%d').date(), Decimal(value))
                for date, value in reader
                if value != '.'
            ]

In [33]:
forex = Forex(file_name)

In [34]:
forex.data

[(datetime.date(2015, 4, 3), Decimal('1.0990')),
 (datetime.date(2015, 4, 6), Decimal('1.1008')),
 (datetime.date(2015, 4, 7), Decimal('1.0850')),
 (datetime.date(2015, 4, 8), Decimal('1.0818')),
 (datetime.date(2015, 4, 9), Decimal('1.0671')),
 (datetime.date(2015, 4, 10), Decimal('1.0598')),
 (datetime.date(2015, 4, 13), Decimal('1.0582')),
 (datetime.date(2015, 4, 14), Decimal('1.0672')),
 (datetime.date(2015, 4, 15), Decimal('1.0596')),
 (datetime.date(2015, 4, 16), Decimal('1.0742')),
 (datetime.date(2015, 4, 17), Decimal('1.0780')),
 (datetime.date(2015, 4, 20), Decimal('1.0763')),
 (datetime.date(2015, 4, 21), Decimal('1.0758')),
 (datetime.date(2015, 4, 22), Decimal('1.0729')),
 (datetime.date(2015, 4, 23), Decimal('1.0803')),
 (datetime.date(2015, 4, 24), Decimal('1.0876')),
 (datetime.date(2015, 4, 27), Decimal('1.0892')),
 (datetime.date(2015, 4, 28), Decimal('1.0979')),
 (datetime.date(2015, 4, 29), Decimal('1.1174')),
 (datetime.date(2015, 4, 30), Decimal('1.1162')),
 (dat

Now, there's one more thing we could do to make the data easier to work with. Notice how the data is just a list of tuples.

So to access the date in the first row, we would have to write:

In [35]:
forex.data[0][0]

datetime.date(2015, 4, 3)

and for the value:

In [36]:
forex.data[0][1]

Decimal('1.0990')

Would it not be better if we could reference those values using an attribute name, like `date` and `value`?

To do that we are going to create a very simple class:

In [37]:
class DataPoint:
    def __init__(self, date, value):
        self.date = date
        self.value = value

And now, instead of creating a list of tuples for our data, we are going to create a list of instances of `DataPoint`:

In [38]:
class Forex:
    def __init__(self, file_name):
        with open(file_name) as f:
            reader = csv.reader(f)
            
            self.headers = next(reader)
            self.data = [
                DataPoint(datetime.strptime(date, '%Y-%m-%d').date(), Decimal(value))
                for date, value in reader
                if value != '.'
            ]

In [39]:
forex = Forex(file_name)

In [40]:
forex.data[0].date

datetime.date(2015, 4, 3)

In [41]:
forex.data[0].value

Decimal('1.0990')

Let's restructure our code a bit - best practice is to try and keep functions small, including the `__init__` method.

So, we don't have to parse the `date` and `Decimal` in the `__init__`, we could pass the strings to the `DataPoint` class and parse them there instead.

In [42]:
class DataPoint:
    def __init__(self, date, value):
        self.date = datetime.strptime(date, '%Y-%m-%d').date()
        self.value = Decimal(value)
        
class Forex:
    def __init__(self, file_name):
        with open(file_name) as f:
            reader = csv.reader(f)
            self.headers = next(reader)
            self.data = [DataPoint(date, value) for date, value in reader if value != '.']

In [43]:
forex = Forex(file_name)

In [44]:
forex.data[0].date

datetime.date(2015, 4, 3)

In [45]:
forex.data[0].value

Decimal('1.0990')

Also, we don't really need the headers in our `Forex` class, since we have named our `date` and `value` attributes:

In [46]:
class DataPoint:
    def __init__(self, date, value):
        self.date = datetime.strptime(date, '%Y-%m-%d').date()
        self.value = Decimal(value)
        
class Forex:
    def __init__(self, file_name):
        with open(file_name) as f:
            reader = csv.reader(f)
            next(reader)  # skip header row
            self.data = [DataPoint(date, value) for date, value in reader if value != '.']

In [47]:
forex = Forex(file_name)

In [48]:
forex.data[0].date

datetime.date(2015, 4, 3)

Finally, I'd prefer not to process the file directly within the `__init__` method - I prefer to keep that method as clean as possible.

So, we're going to create another instance method that will do the processing. We'll call that method from inside `__init__` and assign the results to `data`:

In [49]:
class DataPoint:
    def __init__(self, date, value):
        self.date = datetime.strptime(date, '%Y-%m-%d').date()
        self.value = Decimal(value)
        
class Forex:
    def __init__(self, file_name):
        self.file_name = file_name
        self.data = self.process_data()  # note that we need to call process_data bound to self!
        
    def process_data(self):
        with open(self.file_name) as f:
            reader = csv.reader(f)
            next(reader)  # skip header row
            return [DataPoint(date, value) for date, value in reader if value != '.']

In [50]:
forex = Forex(file_name)

In [51]:
forex.data[0].date

datetime.date(2015, 4, 3)

We could also list out each item in data:

In [52]:
for item in forex.data[:5]:
    print(item.date, item.value)

2015-04-03 1.0990
2015-04-06 1.1008
2015-04-07 1.0850
2015-04-08 1.0818
2015-04-09 1.0671


Or, we could even do this:

In [53]:
list(forex.data[:5])

Well, that's not very helpful!!

What we're seeing here is Python's default representation of our custom object.

It would be much nicer if we could see printed out the date and value instead.

Coming up in the next set of videos!